## Load protobuf models
To build the protobuf files, I ran:
```
cd src;
touch generated/nanoswap/__init__.py
touch generated/nanoswap/enum/__init__.py
protoc --python_out=generated --proto_path=generated schemas/nanoswap/message/*.proto schemas/nanoswap/enum/*.proto
```

In [1]:
import sys
from google.protobuf.json_format import MessageToJson
sys.path.append("./generated")

import nanoswap.enum.issuers_pb2 as issuers_pb2
import nanoswap.message.identity_pb2 as identity_pb2

## Create an example data record

In [2]:
identity = identity_pb2.Identity(
    id_field_type = issuers_pb2.Issuer.UNITED_STATES_AMERICA___FEDERAL___SOCIAL_SECURITY_NUMBER,
    id_field_content = "123-45-6789"
)

In [3]:
# Note: if the value of id_field_type is set to 1 (default value) it will not be included in the payload
MessageToJson(identity, including_default_value_fields=True)

'{\n  "idFieldContent": "123-45-6789",\n  "idFieldType": "UNITED_STATES_AMERICA___FEDERAL___SOCIAL_SECURITY_NUMBER"\n}'

## Serialize the example record to prepare to store it in ipfs

In [4]:
content = identity.SerializeToString()
content

b'\x12\x0b123-45-6789'

## Try to deserialize it

In [5]:
reader = identity_pb2.Identity()
reader.ParseFromString(content)
MessageToJson(reader, including_default_value_fields=True)

'{\n  "idFieldContent": "123-45-6789",\n  "idFieldType": "UNITED_STATES_AMERICA___FEDERAL___SOCIAL_SECURITY_NUMBER"\n}'

## Write the serialize message to ipfs

In [6]:
import uuid

# suppose this is the new credit identity we decided to issue for the user
credit_id = uuid.uuid4()
credit_id

UUID('86ff683e-ca37-4b3c-9871-67ac40641107')

In [7]:
# uuid4 type conversion examples
print(credit_id.bytes)
print(str(credit_id))

b'\x86\xffh>\xca7K<\x98qg\xac@d\x11\x07'
86ff683e-ca37-4b3c-9871-67ac40641107


In [8]:
filename = 'identity.' + str(credit_id)
filename

'identity.86ff683e-ca37-4b3c-9871-67ac40641107'

## Write the file that we want to upload
Long term it would be better to use HTTP or somehow write it in memory, but this looked like the easiest method

In [9]:
filepath = "generated/tmp/" + filename
with open(filepath, "wb") as f:
    f.write(content)

## Upload the file to ipfs with the cli

In [10]:
import subprocess

In [27]:
result = subprocess.run(["ipfs", "add", filepath, "--pin=true", "--to-files", "/" + filename, "-Q"], capture_output=True)
hash = result.stdout.strip()
hash

b'QmUYz8RCMdcbaekmpGRNdDkVyfVghZwkoKr9d7KydFXQwP'

## Read the data back from ipfs

In [29]:
result = subprocess.run(["ipfs", "cat", hash], capture_output=True)
data = result.stdout.strip()
data

b'\x12\x0b123-45-6789'

In [30]:
reader = identity_pb2.Identity()
reader.ParseFromString(data)
MessageToJson(reader, including_default_value_fields=True)

'{\n  "idFieldContent": "123-45-6789",\n  "idFieldType": "UNITED_STATES_AMERICA___FEDERAL___SOCIAL_SECURITY_NUMBER"\n}'